# Run this in Google Colab

<a target="_blank" href="https://colab.research.google.com/github/morriswong/hungrydata/blob/main/llm_with_search.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [32]:
%%capture
%pip install -qU langchain-groq
%pip install -qU langchain
%pip install -qU langchain-community
%pip install -qU duckduckgo-search

In [33]:
import os
from langchain_groq import ChatGroq
from google.colab import userdata

os.environ["GROQ_API_KEY"] = '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'

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [34]:
from langchain.agents import initialize_agent
from langchain_community.tools import Tool, DuckDuckGoSearchRun, DuckDuckGoSearchResults
from langchain.agents import Tool

ddg_search = DuckDuckGoSearchResults()

tools = [
   Tool(
       name="DuckDuckGo Search",
       func=ddg_search.run,
       description="Useful to browse information from the Internet.",
   )
]

agent = initialize_agent(
   tools, llm, agent="zero-shot-react-description", verbose=True
)

In [38]:
from langchain.agents import initialize_agent

agent = initialize_agent(
   tools, llm, agent="zero-shot-react-description", verbose=True
)

agent.run(
   "top 10 competitors  of starbucks"
)



> Entering new AgentExecutor chain...
To find the top 10 competitors of Starbucks, I will use the duckduckgo\_results\_json tool to search the internet for this information.

Action: duckduckgo_results_json
Action Input: "top 10 competitors of starbucks"
Observation: snippet: In this article, we'll dive into the top 10 competitors challenging Starbucks' coffee crown in 2024. From international powerhouses to quirky local favorites, get ready for a journey through the diverse landscape of coffee shop rivals. Who's got what it takes to outbrew the Seattle giant? Let's find out! Top Competitors of Starbucks 1., title: Top Starbucks Competitors: Battle for Coffee Supremacy, link: https://thebrandhopper.com/2024/10/04/top-starbucks-competitors-battle-for-coffee-supremacy/, snippet: Starbucks' annual revenue for 2023 was $35.98 billion and net income of 4.12 billion. The company offers a wide range of whole bean and ground coffees, as well as instant coffee. Starbucks has several competito

"The top 10 competitors of Starbucks in 2024 are Dunkin Donuts, Costa Coffee, McDonald's (McCafe), Tim Hortons, Nestle (Nescafe), Keurig Dr Pepper (Green Mountain Coffee), JAB Holding Company (Peet's Coffee, Caribou Coffee, Keurig), The Kraft Heinz Company (Maxwell House), Tchibo, and Caffe Nero."

In [42]:
import os
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain_community.tools import DuckDuckGoSearchResults

# Set up the OpenAI API key
os.environ["OPENAI_API_KEY"] = 'your_openai_api_key'

# Define the custom prompt template
template = """
You are expert analyst and searcher to list down the top 10 competitors of {company_name}.

Requirements:
- analyse the search results for the competitors of the company.
- filter out only the top 10 national level competitors excluding the local ones.
- response expected should be in the below json format only
{"competitors_list":"[list of 10 national competitors]"}
"""

prompt = PromptTemplate(
    input_variables=["company_name"],
    template=template
)

# Initialize the language model
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Initialize the DuckDuckGo search tool
ddg_search = DuckDuckGoSearchResults()

# Define a custom function to process search results
def process_search_results(query: str) -> str:
    """Fetches and processes search results for the given query."""
    results = ddg_search.run(query)
    # Process the results as needed; here, we simply return them
    return results

# Create a Tool from the custom function
custom_tool = Tool(
    name="ProcessSearchResults",
    func=process_search_results,
    description="Fetches and processes search results for a given query."
)

# List of tools to be used by the agent
tools = [custom_tool]

# Initialize the agent with the custom prompt and tools
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# Define the company name
company_name = "Starbucks"

# Run the agent with the custom prompt
response = agent.run(company_name)

# Print the structured output
print(response)




> Entering new AgentExecutor chain...
The user is asking about Starbucks, which could mean they want to know more about the company, its history, its products, or perhaps they're looking for a nearby Starbucks location. I need more information to provide a specific answer. I'll start by conducting a search to gather more information.

Action: ProcessSearchResults
Action Input: Starbucks
Observation: snippet: Starbucks originally opened in Seattle, Washington, on March 30, 1971. [7] By selling high-quality coffee beans and equipments related, Starbucks became a local coffee bean retailer for the first ten years in Seattle. [8] It was founded by business partners Jerry Baldwin, Zev Siegl and Gordon Bowker who first met as students at the University of San Francisco: [9] The trio were inspired to ..., title: Starbucks - Wikipedia, link: https://en.wikipedia.org/wiki/Starbucks, snippet: Starbucks customers can enjoy Starbucks® Sunsera Blend™ in cafés as the blonde roast brewed Coffee of 